In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [27]:
#!pip install -qU langchain-openai
#!pip install -qU langchain-mistralai

In [3]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="open-mistral-7b")
model


ChatMistralAI(client=<httpx.Client object at 0x000001FE486A1C10>, async_client=<httpx.AsyncClient object at 0x000001FE4858C1A0>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model='open-mistral-7b')

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="co robisz wieczorem ja bede jechal a ty pewnie bedziesz jechal konno!"),
]

model.invoke(messages)


#?????

AIMessage(content='In modo italiano, la frase sarebbe: "Tu stai cucinando la notte, io sarò in auto e tu sicuramente sarai in sella a cavallo!"\n\nPer tradurre correttamente, ho fatto riferimento al contesto di una conversazione tra due persone, in cui si parla di dover andare in diversi luoghi durante la notte. "Co robisz wieczorem" significa "cosa stai facendo la notte", mentre "ja bede jechal a ty pewnie bedziesz jechal konno" significa "io sarò in auto e tu sicuramente sarai in sella a cavallo".\n\nLa traduzione corretta sarebbe quindi "Tu stai cucinando la notte, io sarò in auto e tu sicuramente sarai in sella a cavallo".\n\nIn generale, "ja bede jechal" significa "io sarò in auto", mentre "ty pewnie bedziesz jechal konno" significa "tu sicuramente sarai in sella a cavallo".', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 39, 'total_tokens': 300, 'completion_tokens': 261}, 'model': 'open-mistral-7b', 'finish_reason': 'stop'}, id='run-30a8bd81-cf20-421a-

In [50]:
model.invoke([HumanMessage(content="Kiedy jezyk wloski zmienil sie z laciny?")])

AIMessage(content='Język włoski rozwinął się z języka łacińskiego w okresie średniowiecza, na początku IX w. W tym czasie język łaciński był używany przez kleryków i szlachtę, a język włoski był językiem mieszkańców wsi i miast. Z czasem język włoski rozwinął się i rozszerzył swoje użycie, ale jeszcze w XIII w. był on pod wpływem języka łacińskiego. Jednakże w XIV w. rozpoczęto proces standardyzacji języka włoskiego, który w końcu doprowadził do jego odróżnienia od języka łacińskiego.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 20, 'total_tokens': 225, 'completion_tokens': 205}, 'model': 'open-mistral-7b', 'finish_reason': 'stop'}, id='run-28f606b4-359e-4504-8b91-620ed4bf6ba5-0', usage_metadata={'input_tokens': 20, 'output_tokens': 205, 'total_tokens': 225})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="Your name is Bob, as you mentioned at the beginning of our conversation. Is there anything specific you would like to know or discuss related to your name or any other topic? I'm here to help!", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 28, 'total_tokens': 70, 'completion_tokens': 42}, 'model': 'open-mistral-7b', 'finish_reason': 'stop'}, id='run-638a140e-5c2a-40c6-b174-26a6d72dce30-0', usage_metadata={'input_tokens': 28, 'output_tokens': 42, 'total_tokens': 70})

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

In [8]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Bob! How can I assist you today? Is there a specific topic or question you'd like to discuss? I'm here to help with answers, explain concepts, or just chat about interesting topics. Let me know what you'd like to talk about!


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Based on our conversation, your name is Bob. Is there anything else you'd like to talk about or any specific topic you're interested in? I'm here to help with any questions you have!


In [10]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, I don't have that information. You haven't provided your name in our conversation. Could you please tell me?


In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [12]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Gianlucca."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arr matey Gianlucca, me hearty! I be answerin' yer questions as best as me parrot can squawk it out. Now what be ye askin' for, landlubber?


In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [15]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Luca."
language = "Italian"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ciao Luca! Come posso aiutarti oggi?


In [17]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Sei Luca.


In [18]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

c:\Users\mikoo\miniconda3\envs\primo\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [21]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()